## Youtube API - Parte 2 - Recuperar métricas dos vídeos

Iremos recuperar as métricas dos videos previamente capturados.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('50_videos_copa_mundo.csv', sep=';', encoding='utf-8')

In [ ]:
df

In [ ]:
len(df.videoId.unique())

In [ ]:
len(df)

Será necessário utilizar a mesma chave anteriormente utilizada para acessar a API.

In [ ]:
from apiclient.discovery import build

In [ ]:
nome_servico = "youtube"
versao = "v3"
chave = ""

In [ ]:
api = build(nome_servico, versao, developerKey=chave)

In [ ]:
df_aux = df.copy()

### Recuperar as métricas dos videos

Iremos criar uma função para facilitar a recuperação das métricas por ```videoId```. 

Essa função recebe o id do vídeo e recupera as métricas. Caso não exista ou algo errado aconteça, retorna None.

In [ ]:
def recuperar_metricas(api, id_video):
    metricas = api.videos().list(
        id=id_video,
        part='statistics'
    ).execute()
    try:
        return(metricas['items'][0]['statistics'])
    except:
        return None

Testando a função

In [ ]:
tipos_metricas = recuperar_metricas(api, df_aux.videoId[1])

In [ ]:
print(tipos_metricas)

Iremos adicionar essas métricas no dataframe criando uma coluna para cada métrica. Iremos popular as colunas com o valor 0.

In [ ]:
for item in tipos_metricas:
    df_aux[item] = 0

In [ ]:
df_aux.head()

In [ ]:
from time import time

In [ ]:
# Define o tempo inicial
ti = time()

# Para cada video (linha) no DF
for i in range(len(df_aux)):
    # Executa a função para recuperar as métricas
    metricas = recuperar_metricas(api, df_aux.videoId[i])
    
    # Se exisitir as métricas
    if metricas:
        # Para cada métrica
        for m in metricas:
            try:
                # Recupera a linha (i) e a coluna (m) e atribui o valor que foi recuperado pela métrica.
                df_aux.loc[i, m] = metricas[m]
            # Caso aconteça algum erro
            except Exception as e:
                print(e)
                pass

# Define o tempo final da execução
tf = time() - ti
print("Demorou {} segundos".format(tf))

Para facilitar os cálculos iremos converter as colunas para o tipo numérico.

In [ ]:
df_final = df_aux.copy()

In [ ]:
for m in tipos_metricas:
    df_final[m] = df_final[m].apply(pd.to_numeric)

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
df_final.describe()

In [ ]:
df_final.to_csv('metricas_videos.csv', sep=';', header=True, index=False, encoding='utf-8')

In [ ]:
df_final

### Exercícios - Realizando algumas análises

#### Quais os 5 vídeos mais visualizados?


In [ ]:
top5_visualizados = df_final.viewCount.sort_values(ascending=False)[:5]
top5_visualizados

In [ ]:
top5_visualizados.index

In [ ]:
df_final.loc[top5_visualizados.index]

#### Quais os 5 vídeos mais comentados?

In [ ]:
top5_comentados = df_final.commentCount.sort_values(ascending=False)[:5]
top5_comentados

In [ ]:
df_final.loc[top5_comentados.index]

#### Qual é o vídeo com mais dislike?

In [ ]:
mais_dislike = df_final.dislikeCount.sort_values(ascending=False)[:1]

In [ ]:
mais_dislike

In [ ]:
print(df_final.loc[mais_dislike.index].title.loc[mais_dislike.index])